# Sistemas de recomendación

El **cold-start problem** se trata de recomendar productos a clientes de los cuáles no se tiene ninguna información. Una manera de resolver es recomendar los productos más populares. La siguiente es una **interaction matrix** que representa la interacción entre los clientes y los productos.

In [1]:
import pandas as pd 
import numpy as np 
interaction=pd.read_csv('https://bradfordtuckfield.com/purchasehistory1.csv') 
interaction.set_index("Unnamed: 0", inplace = True) 
interaction

,user1,user2,user3,user4,user5
Unnamed: 0,,,,,
item1,1,1,0,1,1
item2,1,0,1,1,0
item3,1,1,0,1,1
item4,1,0,1,0,1
item5,1,1,0,0,1


In [2]:
interaction_withcounts=interaction.copy()
interaction_withcounts.loc[:,'counts']=interaction_withcounts.sum(axis=1)
interaction_withcounts=interaction_withcounts.sort_values(by='counts',ascending=False)
list(interaction_withcounts.index)

['item1', 'item3', 'item2', 'item4', 'item5']

## Item-based collaborative filtering

Si tenemos información acerca de un cliente, particularmente, su **interés en el producto x**, podemos revisar el historial de productos que frecuentemete se compran conjuntamente con el producto x. Esta técnica se llama item-based collaborative filtering.

### Similitud de vectores

Cada renglón de la matriz de interacciones puede interpretarse como un vector, Medimos la similitud entre vectores mediande el ángulo entre ellos. El calculo se denomina **cosine similarity**.

In [3]:
# Producto punto
def dot_product(vector1,vector2): 
    thedotproduct=np.sum([vector1[k]*vector2[k] for k in range(0,len(vector1))]) 
    return(thedotproduct)

In [4]:
# Magnitud
def vector_norm(vector): 
    thenorm=np.sqrt(dot_product(vector,vector)) 
    return(thenorm)

In [5]:
 # Cosine similarity
 def cosine_similarity(vector1,vector2): 
    thedotproduct=dot_product(vector1,vector2) 
    thecosine=thedotproduct/(vector_norm(vector1)*vector_norm(vector2)) 
    thecosine=np.round(thecosine,4) 
    return(thecosine)

### Implementación

In [6]:
ouritem='item1' 
otherrows=[rowname for rowname in interaction.index if rowname!=ouritem] # Filtrar nombres
otheritems=interaction.loc[otherrows,:] # Filtrar dataframe
theitem=interaction.loc[ouritem,:] # Filtrar item1

In [7]:
similarities=[] 
for items in otheritems.index: 
    similarities.append(cosine_similarity(theitem,otheritems.loc[items,:])) 
otheritems['similarities']=similarities 
recommendations = list(otheritems.sort_values(by='similarities',ascending=False).index)

In [8]:
recommendations

['item3', 'item5', 'item2', 'item4']

La siguiente función devuelve la lista de recomendaciones para un item dado:

In [9]:
def get_item_recommendations(interaction,itemname): 
    otherrows=[rowname for rowname in interaction.index if rowname!=itemname] 
    otheritems=interaction.loc[otherrows,:] 
    theitem=list(interaction.loc[itemname,:]) 
    similarities=[] 
    for items in otheritems.index: 
        similarities.append(cosine_similarity(theitem,list(otheritems.loc[items,:]))) 
    otheritems['similarities']=similarities 
    return list(otheritems.sort_values(by='similarities',ascending=False).index)

In [10]:
get_item_recommendations(interaction,'item2')

['item4', 'item1', 'item3', 'item5']

## User-based collaborative filtering

El enfoque-usuario interpreta las *compras de usuarios* como vectores, y compara los usuarios más similares para recomendar los productos. Por ejemplo, la similitud entre los usuarios 2 y 5:

In [11]:
user2=interaction.loc[:,'user2'] 
user5=interaction.loc[:,'user5'] 
cosine_similarity(user2,user5)

0.866

Reajustamos la función para comparar usuarios:

In [12]:
def get_similar_users(interaction,username): 
    othercolumns=[columnname for columnname in interaction.columns if columnname!=username] 
    otherusers=interaction[othercolumns] 
    theuser=list(interaction[username]) 
    similarities=[] 
    for users in otherusers.columns: 
        similarities.append(cosine_similarity(theuser,list(otherusers.loc[:,users]))) 
    otherusers.loc['similarities',:]=similarities 
    return list(otherusers.sort_values(by='similarities',axis=1,ascending=False).columns)

La siguiente función recomienda todos los productos que ha comprado el usuario más similar que el usuario analizado no ha comprado: 

In [13]:
def get_user_recommendations(interaction,username): 
    similar_users=get_similar_users(interaction,username) 
    purchase_history=interaction[similar_users[0]] 
    purchased=list(purchase_history.loc[purchase_history==1].index) 
    purchased2=list(interaction.loc[interaction[username]==1,:].index) 
    recs=sorted(list(set(purchased) - set(purchased2))) 
    return(recs)

In [14]:
get_user_recommendations(interaction,'user2')

/tmp/ipykernel_90/3264557.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  otherusers.loc['similarities',:]=similarities


['item4']

## Caso de estudio: música

In [15]:
import pandas as pd
lastfm = pd.read_csv("https://bradfordtuckfield.com/lastfm-matrix-germany.csv")
lastfm.head()

,user,a perfect circle,abba,ac/dc,adam green,aerosmith,afi,air,alanis morissette,alexisonfire,...,timbaland,tom waits,tool,tori amos,travis,trivium,u2,underoath,volbeat,yann tiersen
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,33,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,42,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,51,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,62,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
lastfm.drop(['user'],axis=1,inplace=True)

Trasponemos el dataframe para que los renglones representen artistas y las columnas representen usuarios:

In [17]:
lastfmt=lastfm.T

In [18]:
lastfmt.shape

(285, 1257)

Vemos los 10 artistas recomendados relacionados con el artista 'abba':

In [19]:
get_item_recommendations(lastfmt,'abba')[0:10]

['madonna',
 'robbie williams',
 'elvis presley',
 'michael jackson',
 'queen',
 'the beatles',
 'kelly clarkson',
 'groove coverage',
 'duffy',
 'mika']

Aquí se recomiendan 3 artistas según el enfoque user-based para el usuario 0:

In [20]:
get_user_recommendations(lastfmt,0)[0:3]

/tmp/ipykernel_90/4050012869.py:4: RuntimeWarning: invalid value encountered in double_scalars
  thecosine=thedotproduct/(vector_norm(vector1)*vector_norm(vector2))
/tmp/ipykernel_90/3264557.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  otherusers.loc['similarities',:]=similarities


['billy talent', 'bob marley', 'die toten hosen']

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=25415bd8-24a1-4217-9df7-438e1e208889' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>